# Reproducing speedy gradient check bug 

In [1]:
from jax.test_util import check_vjp, check_jvp
import functools
import jax
from jcm.model import Model

In [2]:
# Create model that goes through one timestep
model = Model()
state = model._prepare_initial_modal_state()

def f(state_f):
    _ = model.run(total_time=0) # to set up model fields
    predictions = model.run(initial_state=state_f, save_interval=(1/48.), total_time=(1/48.))
    return model._final_modal_state, predictions

# Calculate gradient
f_jvp = functools.partial(jax.jvp, f)
f_vjp = functools.partial(jax.vjp, f) 

check_vjp(f, f_vjp, args = (state,), 
                        atol=None, rtol=1, eps=0.00001)
check_jvp(f, f_jvp, args = (state,), 
                        atol=None, rtol=1, eps=0.001)

AttributeError: 'numpy.ndarray' object has no attribute 'at'